In [28]:
import pandas as pd
import numpy as np
import xgboost as xgb
import mlflow
import dagshub

# Initialize DagsHub + MLflow
dagshub.init(repo_owner='TomC333', repo_name='ml-walmart-recruiting', mlflow=True)
model_name = "Best_XGBoost_Model"
model_uri = f"models:/{model_name}/latest"

# Load the model from MLflow registry
model = mlflow.xgboost.load_model(model_uri)

# Load train and test merged data (with sales)
train_df = pd.read_csv('train_merged_full.csv', parse_dates=['Date'])
test_df = pd.read_csv('data/test.csv', parse_dates=['Date'])
features_df = pd.read_csv('data/features.csv', parse_dates=['Date'])
stores_df = pd.read_csv('data/stores.csv')

# Merge test data with features and stores
test_merged = test_df.merge(features_df, on=['Store', 'Date'], how='left')
test_merged = test_merged.merge(stores_df, on='Store', how='left')

if 'IsHoliday_y' in test_merged.columns:
    test_merged.drop(['IsHoliday_y'], axis=1, inplace=True)
    test_merged.rename(columns={'IsHoliday_x': 'IsHoliday'}, inplace=True)

# Basic date features
test_merged['Month'] = test_merged['Date'].dt.month
test_merged['Year'] = test_merged['Date'].dt.year
test_merged['Week'] = test_merged['Date'].dt.isocalendar().week.astype(int)
test_merged['Quarter'] = test_merged['Date'].dt.quarter

# Sort train and test data
train_df = train_df.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)
test_merged = test_merged.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

# Initialize lag columns in test set as NA
for lag in [1, 2, 3, 4, 52]:
    test_merged[f'Sales_Lag_{lag}'] = pd.NA

# Calculate median sales per Store-Dept for fallback lag filling
median_sales = train_df.groupby(['Store', 'Dept'])['Weekly_Sales'].median()

# Combine train + test for iterative prediction and lag update
combined = pd.concat([train_df, test_merged], sort=False).reset_index(drop=True)

all_predictions = []
unique_dates = sorted(test_merged['Date'].unique())

features = [
    'Store', 'Dept', 'IsHoliday', 'Month', 'Year', 'Week', 'Quarter',
    'Temperature', 'Fuel_Price', 'Size', 'Type',
    'Is_SuperBowl', 'Is_LaborDay', 'Is_Thanksgiving', 'Is_Christmas',
    'Sales_Lag_1', 'Sales_Lag_2', 'Sales_Lag_3', 'Sales_Lag_4', 'Sales_Lag_52'
]

for current_date in unique_dates:
    # Filter test rows for current_date without predicted sales
    test_current = combined[
        (combined['Date'] == current_date) & 
        (combined['Weekly_Sales'].isna())
    ].copy()
    
    # Merge lag features for each lag from previous weeks' sales
    for lag in [1, 2, 3, 4, 52]:
        lag_date = current_date - pd.Timedelta(weeks=lag)
        lag_vals = combined[
            combined['Date'] == lag_date
        ][['Store', 'Dept', 'Weekly_Sales']].copy()
        
        lag_vals.rename(columns={'Weekly_Sales': f'Sales_Lag_{lag}'}, inplace=True)
        
        test_current = test_current.merge(
            lag_vals, on=['Store', 'Dept'], how='left', suffixes=('', f'_lag{lag}')
        ).copy()
        
        lag_col_with_suffix = f'Sales_Lag_{lag}_lag{lag}'
        if lag_col_with_suffix in test_current.columns:
            test_current[f'Sales_Lag_{lag}'] = test_current[lag_col_with_suffix]
            test_current.drop(columns=[lag_col_with_suffix], inplace=True)

    # Fill missing lag values using median fallback per Store-Dept
    for lag in [1, 2, 3, 4, 52]:
        test_current[f'Sales_Lag_{lag}'] = test_current[f'Sales_Lag_{lag}'].fillna(
            test_current.apply(lambda row: median_sales.get((row['Store'], row['Dept']), 0), axis=1)
        )
    
    # Prepare features for prediction
    X_pred = test_current[features].copy()
    
    # Encode 'Type' as category codes
    X_pred['Type'] = X_pred['Type'].astype('category').cat.codes

    # Create DMatrix for XGBoost
    dmatrix_pred = xgb.DMatrix(X_pred)
    
    # Predict sales and clip negatives to zero
    preds = model.predict(dmatrix_pred).clip(min=0)
    
    # Store predictions back in combined dataframe
    combined.loc[test_current.index, 'Weekly_Sales'] = preds
    
    # Save predictions for this date
    all_predictions.append(test_current.assign(Weekly_Sales=preds))

# Combine all date predictions into one DataFrame
predictions_df = pd.concat(all_predictions).sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

# Create 'Id' in predictions_df to match submission Id format
predictions_df['Id'] = (
    predictions_df['Store'].astype(str) + '_' +
    predictions_df['Dept'].astype(str) + '_' +
    predictions_df['Date'].dt.strftime('%Y-%m-%d')
)

# Load submission template
submission = pd.read_csv('data/sampleSubmission.csv')

# Merge predictions on 'Id'
submission = submission.merge(
    predictions_df[['Id', 'Weekly_Sales']],
    on='Id',
    how='left'
)

print(submission)

submission['Weekly_Sales'] = submission['Weekly_Sales_y'].fillna(0)

# Drop the old columns
submission = submission.drop(columns=['Weekly_Sales_x', 'Weekly_Sales_y'])

# Save to CSV
submission.to_csv('submission.csv', index=False)

print("Final submission saved.")
# Save final submission
submission.to_csv('submission.csv', index=False)

print("Inference complete. submission.csv saved.")


Initialized MLflow to track repo "TomC333/ml-walmart-recruiting"

Repository TomC333/ml-walmart-recruiting initialized!

/var/folders/91/01r863q91sz9mhh3gsvm_d640000gn/T/ipykernel_20802/415137199.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat([train_df, test_merged], sort=False).reset_index(drop=True)


                      Id  Weekly_Sales_x  Weekly_Sales_y
0         1_1_2012-11-02               0    39319.292969
1         1_1_2012-11-09               0    30041.716797
2         1_1_2012-11-16               0    29517.824219
3         1_1_2012-11-23               0    16485.263672
4         1_1_2012-11-30               0    20329.417969
...                  ...             ...             ...
115059  45_98_2013-06-28               0      789.463318
115060  45_98_2013-07-05               0      813.141113
115061  45_98_2013-07-12               0      668.398193
115062  45_98_2013-07-19               0      640.965027
115063  45_98_2013-07-26               0      694.878174

[115064 rows x 3 columns]
Final submission saved.
Inference complete. submission.csv saved.
